In [1]:
from pyaci import Node, options
import logging

# Establish a handle to the APIC, and login.
apic = Node('https://praveek6-bld.insieme.local:7000',
            verify=False, disableWarnings=True)
apic.methods.Login('admin', 'ins3965!').POST()

<Response [200]>

In [2]:
# Delete a tenant.
# This is here to cleanup the test tenant, and start from clean slate.
# Don't worry if you're lost a little bit. Follow rest of the examples.
apic.mit.polUni().fvTenant('test').DELETE()

<Response [200]>

In [3]:
# Create a local reference to a test tenant, and a test network.
mit = apic.mit
mit.polUni().fvTenant('test').fvCtx('test')

# Dump the XML representation of the local MIT.
print 'XML of the local MIT cache:'
print mit.Xml

# Dump the JSON representation of the local MIT.
print 'JSON of the local MIT cache:'
print mit.Json

XML of the local MIT cache:
<topRoot>
  <polUni>
    <fvTenant name="test">
      <fvCtx name="test"/>
    </fvTenant>
  </polUni>
</topRoot>

JSON of the local MIT cache:
{
  "topRoot": {
    "children": [
      {
        "polUni": {
          "children": [
            {
              "fvTenant": {
                "attributes": {
                  "name": "test"
                },
                "children": [
                  {
                    "fvCtx": {
                      "attributes": {
                        "name": "test"
                      }
                    }
                  }
                ]
              }
            }
          ]
        }
      }
    ]
  }
}


In [4]:
# Commit the local reference/cache to APIC.
mit.POST(format='xml')

<Response [200]>

In [5]:
# Fetch the management tenant from APIC.
tenant = apic.mit.polUni().fvTenant('mgmt')
tenant.GET()
print tenant.Xml

<fvTenant dn="uni/tn-mgmt" lcOwn="local" ownerKey="" name="mgmt" descr="" status="" monPolDn="uni/tn-common/monepg-default" modTs="2015-05-25T20:47:56.553+00:00" ownerTag="" childAction="" uid="0"/>



In [6]:
# Fetch the in-band BD subtree from APIC.
bd = apic.mit.polUni().fvTenant('mgmt').fvBD('inb')
bd.GET(**options.subtree)
print bd.Xml

<fvBD dn="uni/tn-mgmt/BD-inb" uid="0" arpFlood="no" seg="16383902" unicastRoute="yes" unkMcastAct="flood" descr="" llAddr="::" monPolDn="uni/tn-common/monepg-default" modTs="2015-05-25T20:49:06.353+00:00" scope="2850816" status="" bcastP="225.0.102.80" mac="00:22:BD:F8:19:FF" epMoveDetectMode="" ownerTag="" childAction="" lcOwn="local" ownerKey="" name="inb" unkMacUcastAct="proxy" multiDstPktAct="bd-flood" limitIpLearnToSubnets="no" mtu="inherit" pcTag="16387">
  <fvRsIgmpsn dn="uni/tn-mgmt/BD-inb/rsigmpsn" lcOwn="local" tnIgmpSnoopPolName="" tDn="uni/tn-common/snPol-default" rType="mo" stateQual="default-target" state="formed" modTs="2015-05-25T20:47:56.585+00:00" tCl="igmpSnoopPol" forceResolve="yes" status="" tContextDn="" tRn="snPol-default" uid="0" tType="name" childAction="" monPolDn="uni/tn-common/monepg-default"/>
  <fvRsBDToNdP dn="uni/tn-mgmt/BD-inb/rsBDToNdP" lcOwn="local" tDn="uni/tn-common/ndifpol-default" rType="mo" stateQual="default-target" state="formed" modTs="2015-05

In [7]:
# Query all private networks.
ctxs = apic.mit.GET(**options.subtreeClass('fvCtx'))
for ctx in ctxs:
    print ctx.name

default
test
overlay-1
oob
inb


In [8]:
# Query all private networks under management tenant.
ctxs = apic.mit.polUni().fvTenant('mgmt').GET(**options.subtreeClass('fvCtx'))
for ctx in ctxs:
    print ctx.name

oob
inb


In [9]:
# Query all direct children of management tenant.
for child in apic.mit.polUni().fvTenant('mgmt').GET(**options.children):
    print child.Dn

uni/tn-mgmt/domain-mgmt
uni/tn-mgmt/BD-inb
uni/tn-mgmt/ctx-oob
uni/tn-mgmt/ctx-inb
uni/tn-mgmt/rsTenantMonPol
uni/tn-mgmt/extmgmt-default
uni/tn-mgmt/mgmtp-default


In [10]:
# Query entire subtree under management tenant.
for child in apic.mit.polUni().fvTenant('mgmt').GET(**options.subtree):
    print child.Dn

uni/tn-mgmt/domain-mgmt
uni/tn-mgmt/BD-inb/rsBDToNdP
uni/tn-mgmt/BD-inb/rsbdToEpRet
uni/tn-mgmt/BD-inb/rsctx
uni/tn-mgmt/BD-inb/rsigmpsn
uni/tn-mgmt/BD-inb
uni/tn-mgmt/ctx-oob/rsbgpCtxPol
uni/tn-mgmt/ctx-oob/rsctxToEpRet
uni/tn-mgmt/ctx-oob/rsctxToExtRouteTagPol
uni/tn-mgmt/ctx-oob/rsospfCtxPol
uni/tn-mgmt/ctx-oob/rtmgmtOoBCtx-[uni/tn-mgmt/mgmtp-default/oob-default]
uni/tn-mgmt/ctx-oob/any
uni/tn-mgmt/ctx-oob
uni/tn-mgmt/ctx-inb/rsbgpCtxPol
uni/tn-mgmt/ctx-inb/rsctxToEpRet
uni/tn-mgmt/ctx-inb/rsctxToExtRouteTagPol
uni/tn-mgmt/ctx-inb/rsospfCtxPol
uni/tn-mgmt/ctx-inb/rtctx-[uni/tn-mgmt/BD-inb]
uni/tn-mgmt/ctx-inb/any
uni/tn-mgmt/ctx-inb
uni/tn-mgmt/rsTenantMonPol
uni/tn-mgmt/extmgmt-default
uni/tn-mgmt/mgmtp-default/oob-default/rsooBCtx
uni/tn-mgmt/mgmtp-default/oob-default
uni/tn-mgmt/mgmtp-default
uni/tn-mgmt


In [11]:
# Query faults under out-of-band management EPG.
epg = apic.mit.polUni().fvTenant('mgmt').mgmtMgmtP('default').mgmtOoB('default')
for fault in epg.GET(**options.faults):
    print fault.descr

Configuration failed for EPG default due to Not Associated With Management Zone


In [12]:
# Query audit logs for test private network.
ctx = apic.mit.polUni().fvTenant('test').fvCtx('test')
ctx.descr = 'Test 1'
ctx.POST()
ctx.descr = 'Test 2'
ctx.POST()
for change in ctx.GET(**options.auditLogs):
    print change.created, change.descr, change.changeSet

2015-05-27T02:39:22.946+00:00 Ctx test created knwMcastAct:permit, name:test, pcEnfPref:enforced
2015-05-27T02:42:08.016+00:00 Ctx test modified descr (Old: , New: Test 1)
2015-05-27T02:42:08.058+00:00 Ctx test modified descr (Old: Test 1, New: Test 2)
2015-05-27T02:42:36.848+00:00 Ctx test modified descr (Old: Test 2, New: Test 1)
2015-05-27T02:42:36.891+00:00 Ctx test modified descr (Old: Test 1, New: Test 2)
2015-05-27T02:43:02.283+00:00 Ctx test modified descr (Old: Test 2, New: Test 1)
2015-05-27T02:43:02.508+00:00 Ctx test modified descr (Old: Test 1, New: Test 2)
2015-05-27T02:44:39.486+00:00 Ctx test deleted 
2015-05-27T02:45:08.884+00:00 Ctx test created knwMcastAct:permit, name:test, pcEnfPref:enforced
2015-05-27T02:45:09.366+00:00 Ctx test modified descr (Old: , New: Test 1)
2015-05-27T02:45:09.414+00:00 Ctx test modified descr (Old: Test 1, New: Test 2)
2015-05-27T02:45:23.144+00:00 Ctx test deleted 
2015-05-27T02:45:23.264+00:00 Ctx test created knwMcastAct:permit, name:te

In [13]:
# Use filter support to narrow query response.
for node in apic.mit.GET(**options.subtreeClass('fabricNode') &
                         options.filter(options.Eq('fabricNode.role', 'leaf') |
                                        options.Eq('fabricNode.role', 'spine'))):
    print node.name, node.role

leaf1 leaf
spine1 spine
spine2 spine
leaf2 leaf
